# Análisis de sentimientos en el ámbito sociopolítico del Ecuador

## Importar módulos

In [ ]:
from core.login_twitter import login_twitter
from core.dirver_manager import get_driver, close_driver
from scraping.enlaces_scraper import guardar_enlaces_candidatos
from scraping.tweets_scraper import extraer_informacion_tweets
from scraping.comentarios_scraper import extraer_enlaces_comentarios_csvs
from scraping.contenido_comentarios_scraper import minar_texto_comentarios
from preprocessing.consolidar_corpus import consolidar_corpus_csvs
from preprocessing.limpieza_texto import limpiar_y_normalizar_corpus
from preprocessing.unificar_corpus import unificar_y_limpiar
from preprocessing.reestructurar_csv_a_json import reestructurar_csv_a_json_simple
from storage.mongodb_loader import cargar_json_a_mongo
from dotenv import load_dotenv
from pathlib import Path
import os

## Definición de las cadenas de consulta

In [ ]:
candidatos_presidenciales_2025 = [
    "Daniel Noboa", "Luisa González", "Wilson Gómez Vascones", "Jimmy Jairala",
    "Jorge Escala", "Andrea González Náder", "Henry Kronfle", "Carlos Rabascall", 
    "Leonidas Iza", "Iván Saquicela", "Francesco Tabacchi", "Henry Cucalón",
    "Enrique Gómez", "Víctor Araus", "Juan Iván Cueva", "Pedro Granja"
]


palabras_clave = [
    "elecciones 2025"
    # , "presidenciales 2025", "candidato presidencial", "candidatura 2025",
    # "campaña presidencial", "Ecuador 2025", "presidente Ecuador", "voto 2025",
    # "debate presidencial", "elecciones Ecuador"
]

## Configuración del driver

Al usar get_driver logramos que el web driver no se cierre lo que nos permitirá usar el mismo para realizar las acciones de web scraping

In [ ]:
driver = get_driver()

## Llamadas a las variables de entorno

In [ ]:
# 1. Obtenemos el Directorio de Trabajo Actual (donde se ejecuta el Notebook)
#    Si ejecutas desde 'proyecto/webScraping', esto devolverá 'proyecto/webScraping'
current_dir = Path(os.getcwd())

# 2. Subimos un nivel para llegar a la carpeta 'proyecto/' (donde está .env)
#    Esto cambia la ruta de 'proyecto/webScraping' a 'proyecto'
base_dir = current_dir.parent

# 3. Construimos la ruta completa al archivo .env
dotenv_path = base_dir / '.env'

# --- Carga de Variables ---
load_dotenv(dotenv_path=dotenv_path)

# --- Llamada de Variables ---
username = os.getenv("X_USERNAME")
password = os.getenv("X_PASSWORD")
email = os.getenv("X_EMAIL")

## Inicio de sesión

Nos ayudará a mantener iniciada la sesión y evitar varios logeos

In [ ]:
login_twitter(driver, username, password, email)

## Minar enlaces de las publicaciones en base a las cadenas de consulta

La cadena de consulta se compone de la unión de candidatos_presidenciales_2025 y palabras_claves
Por ejemplo: "Daniel Noboa elecciones 2025"
Una ve minados todos los enlaces a las publicaciones que aparecen en esta consulta, se guardarán los enlces en `Daniel_Noboa_enlaces_tweets.csv`, siendo "enlaces_tweets" el subijo para cada csv correspondiente a cada candidato

In [ ]:
guardar_enlaces_candidatos(driver, candidatos_presidenciales_2025, palabras_clave)

## Minar contenido de cada publicación

In [ ]:
extraer_informacion_tweets(driver, "datasets/raw/enlaces_publicaciones_4", "datasets/raw/informacion_publicaciones_4")

## Minar enlaces de los comentarios de cada publicación

In [ ]:
extraer_enlaces_comentarios_csvs(driver, "datasets/raw/informacion_publicaciones_4", "datasets/raw/enlaces_comentarios_publicaciones_4", max_scrolls=100)

## Minar contenido comentarios de cada enlace

In [ ]:
minar_texto_comentarios(driver, "datasets/raw/enlaces_comentarios_publicaciones_1_original_A", "datasets/raw/datos_completos_publicacion", limite=None)

## Cerrar driver

In [ ]:
close_driver(driver)

## Unir CSV candidatos

In [ ]:
consolidar_corpus_csvs("datasets/raw/datos_completos_publicacion", "datasets/raw/corpus_completo/corpus_comentarios_tweets.csv")

## Limpiar CSV

In [ ]:
limpiar_y_normalizar_corpus("datasets/raw/corpus_completo/corpus_comentarios_tweets.csv", "datasets/processed", nombre_archivo_salida="corpus_preprocesado_corregido.csv")

## Unificar Corpus Histórico y Limpieza final
En este paso, se toma el corpus procesado previamente ("viejo") y se une con los nuevos datos recolectados. Además, se aplica una limpieza profunda para eliminar duplicados finales antes de la conversión a JSON.

In [ ]:
# Este paso unifica el corpus histórico (webScraping) con el actual y elimina duplicados
# Genera: datasets/processed/corpus_preprocesado_unificado.csv
unificar_y_limpiar(
    ruta_original_str="datasets/processed/corpus_preprocesado.csv",
    ruta_nuevo_str="datasets/processed/corpus_preprocesado_corregido.csv",
    ruta_salida_final="datasets/processed/corpus_preprocesado_unificado.csv"
)

## Convertir CSV a JSON

In [ ]:
reestructurar_csv_a_json_simple("datasets/processed/corpus_preprocesado_unificado.csv", "datasets/processed/corpus_completo.json")

## Carga de JSON a MongoDB

In [ ]:
print("--- Iniciando carga a Base de Datos ---")
cargar_json_a_mongo(
    ruta_json="datasets/processed/corpus_completo.json"
)